# Welcome to the MSA Python Workshop!
*Created by Noah Ford, Keshav Rangan, Rikhav Shah* <br> <br>
Follow along the slide presentation [here](https://docs.google.com/presentation/d/1XXgfSkwRq66XnAMTpvxen4Twim9cPGW9ULiQjS75e5o/edit#slide=id.g2864534bda8_0_0) to help complete the tasks outlined for you. <br> <br>
In short, we will take you from absolute CS to beginner to CS 389: Machine Learning within one hour! (okay maybe not that far)

## Task 2.2 Understanding CoLab
If you a complete beginner to coding, then CoLab is a great place to start! <br> <br>
You can mix blocks of code and blocks of text!

In [ ]:
"code!"

'code!'

text!

### Task 2.2.1 Run a code block!

In [ ]:
"run me!"

'run me!'

You did it!
Let's do it again!

In [ ]:
# I'm a comment, run me!

That's right, no output. <br>
Comments do not affect code in any way. <br>
They are for us, not the computer.

Ok, one more time!

In [ ]:
# run me!
2023 - 1970

53

That's right, we can do math, and many operations here!

In [ ]:
4 > 3

True

In [ ]:
"i" in "team"

False

In [ ]:
"i" in "win"

True

### Task 2.2.1 Variable Assignment

Go to [*statmuse*](https://www.statmuse.com/nfl/ask/most-road-playoff-wins-by-a-quarterback) and fill out the variables below. <br>
Put the values you find where the #TODO text is.

In [ ]:
tomBradyRoadPlayoffWins = 11
markSanchezRoadPlayoffWins = 4
brettFarveRoadPlayoffWins = 3
markSanchezRoadPlayoffWins > brettFarveRoadPlayoffWins

True

We can see this is essentially the same code as above when we wrote: <br>
4 > 3

## Task 2.3 Installing Packages
Ok, now to the meat of the data scrape! <br>
We need some tools to help us.

In [ ]:
import os
from bs4 import BeautifulSoup
!pip install playwright
!playwright install
!playwright install-deps firefox
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import re
import time
import asyncio
from pathlib import Path
import pandas as pd

## Task 2.4 Initializing Functions
This is a fun step! <br>
Run every cell with one click!

In [ ]:
# function run me
async def get_html(url,selector,sleep =5, retries =3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep *i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [ ]:
# function run me
# sus manipulation of playoff results table to get correct elmination order
def filterTeams(arr):
    teamPgsRev = list(reversed(arr))
    finalTeams = []
    for i in teamPgsRev:
        if i not in finalTeams:
            finalTeams.append(i)
    [finalTeams[0],finalTeams[1]] = [finalTeams[1],finalTeams[0]]
    return finalTeams

In [ ]:
# function run me
def getResult(index):
    if index <= 0 :
      return 5
    elif index <= 1:
      return 4
    elif index <= 3:
      return 3
    elif index <= 7:
      return 2
    else :
      return 1

In [ ]:
# function run me
def getYear(url):
    yr = int(url[url.find(".htm")-4:url.find(".htm")]);
    return yr;

In [ ]:
# function run me
def getSeasonYear(url):
  yr = int(url[url.find("index.htm")-5:url.find("index.htm")-1])
  return yr;

In [ ]:
# function run me
def getAbbrv(url):
    abbrv = url[url.index("teams/")+6:url.index(".htm")-5];
    return abbrv;

In [ ]:
# function run me
def getWins(string):
    w = int(string[:string.find("-")])
    return w;

In [ ]:
# function run me
# sus url manipulation to get the next year
def getNext(url,diff):
    base = url[:url.find(".htm")-5]
    yr = getYear(url)+diff;
    if((yr > 2023) or (yr < 19)):
        return "";
    return f"{base}/{yr}.htm";

In [ ]:
# function run me
def nameTeam(link):
    return getAbbrv(link)+str(getYear(link))+".htm";

In [ ]:
# function run me
#get record from html
async def getRec(link):
    html = await savePath(link,"#meta")
    html = BeautifulSoup(html)
    string = html.find_all("p")[2].getText()
    strR = string[string.find(":")+2:string.find(",")]
    return strR

In [ ]:
cache = [];

In [ ]:
async def savePath(url,tag):
  for arr in cache:
    if url == arr[0] :
      return arr[1];
  ret = cache.append([url,await get_html(url, tag)])

In [ ]:
from pathlib import Path
DATA_DIR = "sbpgs";
YR_DIR = "leagueYrs";
root_path = r"/Users/noahford/Desktop/msaPython/"
yr_path = Path(root_path, "leagueYrs")
# sus function, don't touch right now
async def savePath2(link,directory,name,tag):
    save_path = Path(directory, name(link))
    if not(save_path.exists()):
        html = await get_html(link, tag);
        with open(save_path, "w+") as f:
            f.write(html)
    else :
        with open(save_path, 'r') as f:
            html = f.read()
    return html

def nameYear(link):
    i = link.find("years")+6;
    return link[i:i+4] + "league.htm";
def nameTeam(link):
    return getAbbrv(link)+str(getYear(link))+".htm";

In [ ]:
# function run me
async def scrape_season(season):
    base = f"https://www.pro-football-reference.com";
    url = f"{base}/years/{season}/index.htm"
    html = BeautifulSoup(await savePath(url,"#div_playoff_results"))
    # html = BeautifulSoup(await savePath2(url,yr_path,nameYear,"#div_playoff_results"))
    # html = BeautifulSoup(await get_html(url, "#div_playoff_results"))
    a_tags = html.find_all("a");
    hrefs = [a["href"]  for a in a_tags];
    teams = [l for l in hrefs if "/teams/" in l];
    finalTeams = filterTeams(teams);
    finalTeams = [base + t for t in finalTeams]
    # ret = [[t,getNext(t,1),getAbbrv(t),getYear(t)] for t in finalTeams]
    ret = [[t,getNext(t,1)] for i,t in enumerate(finalTeams)]
    return ret

In [ ]:
season = 2022;
url = "https://www.pro-football-reference.com/years/1999/"

## Task 3.1 Scrape Data

### Task 3.1.4 Call savePath
Now we've found where to look for the html, let's pull it! <br>
Fill in the url and tag you found

In [ ]:
url = #TODO, from task 3.1.1
tag = #TODO, from task 3.1.3

In [ ]:
#run me!
await savePath("https://www.pro-football-reference.com/years/2022/","#div_playoff_results")

### Optional Task 3.1.5: Work with html


In [ ]:
# first, store the html mess into a variable
s22 = await savePath("https://www.pro-football-reference.com/years/2022/","#div_playoff_results")
# then, we use the Beautiful Soup package to make the html look nicer
s22BS = BeautifulSoup(s22)
s22BS;

In [ ]:
# this pulls every a tag, or link of a certain type
a_tags = s22BS.find_all("a")
a_tags

In [ ]:
# now, we want the second to last link
loser = a_tags[len(a_tags) - 2]
loser

<a href="/teams/phi/2022.htm">Philadelphia Eagles</a>

In [ ]:
# but that's not a complete link!
base = "https://www.pro-football-reference.com"
link = base + loser["href"]
link

Now we have our link!
Let's pull the record from this page.

In [ ]:
await getRec(link)

Ah! same error! <br>
But remember, this "TypeError" is okay.
We still pulled the page

In [ ]:
await getRec(link)

There we go!  It's the Eagles record!

### Task 3.1.6 Scrape Season

In [ ]:
# call me!
await scrape_season(2022)

Look at how beautiful that is!

## Task 4.0 Import CSV

For the remainder of the workshop, you will be working with this set of pulled data

In [ ]:
raw = "https://raw.githubusercontent.com/NFordUMass/msa/gh-pages/assets/python/output3.csv"

# here, we are creating a data frame from the csv
df = pd.read_csv(raw)

# df is now the variable storing your dataframe
df

Each row is a playoff team's stats for the year, and the stats for the team the next year.

## Task 4.1 Playing with Data

First, find out what the data looks like!

In [ ]:
df.info()

Here we can see each column name, the number of items, and the data type. <br>
The data type? <br>
You've been working with data types this entire workshop!

In [ ]:
5 == "5"

In [ ]:
type(5)

In [ ]:
type("5")

Above, we see that 5 is not equivalent to "5". <br>
That's because "5" is a string (str), or a series of characters! <br>

Most of the data types are "int64" or "float64", both meaning numbers. <br>
Where it says "object", it usually is a string, or series of characters.

Back to the dataframe, now we want to look at some stats about the data.

In [ ]:
df.describe()

The dataframe is smart, as it only calculates averages on numbers, not strings!

Say you're just a big patriots fan, and you're only interested in looking at Patriot playoff teams.

In [ ]:
#replace #TODO with the full name of the team you want to look at!
df_oneTeam = df.query("Team == 'TODO'") # be sure to keep ' ' around the name, in addition to the " "
display(df_oneTeam);

Now, think about how to find only the super bowl losers... <br>
Below are a couple more examples about how to query into the sheet

In [ ]:
df_greatRegSeason = df.query("Wins > 14")
df_greatRegSeason

In [ ]:
# this gives us only teams that lost in the wild card round
df_wcLosers = df.query("Playoff_Rounds_Won == 1")
df_wcLosers

Hey!  That makes no sense... <br>
A wild card loser wouldn't have won any games! <br>
We need to change the column name...

In [ ]:
# give it a new name in #TODO!
newDF =df.rename(columns = {'Playoff_Rounds_Won':'#TODO'})
newDF

Ok, now, how do we find the super bowl losers?

In [ ]:
# fill the #TODO with a playoff round that gives SB losers
losers = df.loc[df['Playoff_Rounds_Won'] == #TODO]
losers

We did it!! <br>
We can check it's correct because we see the Bills are in there 4 times in a row.

## Task 5.0 Export Data
Save all of the work you just made!


In [ ]:
from google.colab import files
losers.to_csv("sbLosers.csv")
files.download("sbLosers.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>